In [1]:
import csv
import json


In [7]:
#Recupérer les informations du fichiers CSV "organizers-data.csv"
def getdatacsv():
    #Chargement des données du CSV
    with open(csvFilePath) as csvFile:          #Lecture du fichier CSV pour récupérer les informations
        csvReader = csv.DictReader(csvFile)
        data = {}                               #Création d'un dictionnaire pour stocker les données
        for rows in csvReader:
            rows['id'] = int(rows['id'])        #Transformation du type des données
            rows['total ticket number'] = int(rows['total ticket number'])
            rows['maximum tickets per user'] = int(rows['maximum tickets per user'])
            id = rows['id']
            data[id] = rows
        return data

#Recupération des données du fichier json "smart-contracts-data.json"
def getdatajson(x):
    ticketcollectionlist = []                                #Création d'une list pour stocker les différentes collections possible (comme dans l'event 4)
    with open(scjsonFilePath) as json_file:
        datajson = json.load(json_file)
        for i in range (len(datajson)):
            if datajson[i]['event_id'] == x:                 #On récupère tous les évenements correspondant aux events du même id du fichier csv
                ticketcollectionlist.append(datajson[i])
        return ticketcollectionlist

#Création d'une list pour les lineups
def getalllineup(datacsv):                                   
    listlineup = []
    strtemp =""
    for i in range (len(datacsv["line up (optional)"])):
        if datacsv["line up (optional)"][i] == "-":
            listlineup.append(strtemp)
            strtemp = ""
        else:
            strtemp += datacsv["line up (optional)"][i]
    listlineup.append(strtemp)
    return listlineup

#Remplir les informations de la section 'ticketCollections' avec les informations du json
def rempDicTicketCollection(ticketCollectionData, datajson):
    for keys in ticketCollectionData:
        if keys == "collectionName":
            ticketCollectionData[keys] = datajson['collection_name']
        if keys == "scAddress":
            ticketCollectionData[keys] = datajson['smart_contract']['crowdsale']
        if keys == "collectionAddress":
            ticketCollectionData[keys] = datajson['smart_contract']['collection']
        if keys == "pricePerToken":
            ticketCollectionData[keys] = datajson['smart_contract']['sale_params']['price_per_token']
        if keys == "maxMintPerUser":
            ticketCollectionData[keys] = datajson['smart_contract']['sale_params']['max_mint_per_user']
        if keys == "saleSize":
            ticketCollectionData[keys] = datajson['smart_contract']['sale_params']['sale_size']
    return ticketCollectionData

#Recupère une list de ticketcollection. Permet d'ajouter plusieurs ticketcollection à un même event (cf event 4)
def getListOfCollections(ticketCollectionData,datajson):
    listofdatacollection =[]
    for i in range (len(datajson)):
        listofdatacollection.append(rempDicTicketCollection(ticketCollectionData, datajson[i]))
    return listofdatacollection        

#Complète les informations de notre dictionnaire principale avec l'ensemble des informations, ce dictionnaire correspond à UN event complet avec les informations du CSV et du JSON
def remplirDic(workDictionnaire, datacsv,ticketCollectionData,datajson):
    for keys in workDictionnaire:
        if keys == "eventId":
            workDictionnaire[keys] = datacsv["id"]
        if keys == "title":
            workDictionnaire[keys] = datacsv["event title"]    
        if keys == "startDatetime":
            workDictionnaire[keys] = datacsv["event start date"]
        if keys == "endDatetime":
            workDictionnaire[keys] = datacsv["event end date"]
        if keys == "locationName":
            workDictionnaire[keys] = datacsv["name of the location hosting the event (optional)"]
        if keys == "address":
            workDictionnaire[keys] = datacsv["address of the location"]
        if keys == "totalTicketsCount":
            workDictionnaire[keys] = datacsv["total ticket number"]
        if keys == "ticketCollections":
            workDictionnaire[keys] = getListOfCollections(ticketCollectionData,datajson)   
        if keys == "lineUp":
            workDictionnaire[keys] = getalllineup(datacsv)
        if keys == "assetUrl":
            workDictionnaire[keys] = datacsv["event image or video url (mp4 or png or jpeg)"]
    if datacsv["line up (optional)"] == "":
        workDictionnaire.pop('lineUp') #Suppression de la clé 'lineup'
    if datacsv["name of the location hosting the event (optional)"] == "":
        workDictionnaire.pop('locationName')
    return workDictionnaire

#Création du dictionnare principale
def createDictionnaire():
    workDictionnaire = {
        "eventId": 0,
        "title": "",
        "startDatetime": "",
        "endDatetime": "",
        "address": "",
        "locationName": "",
        "totalTicketsCount": 0,
        "assetUrl": "",
        "lineUp": [
        ],
        "ticketCollections": [
        {
  
      }
    ]
  }
    return workDictionnaire

#Création du dictionnaire pour le ticketcollection
def createCollection():
  ticketCollectionData = {
        "collectionName": "",
        "scAddress": "",
        "collectionAddress": "",
        "pricePerToken": 0,
        "maxMintPerUser": 0,
        "saleSize": 0
      }
  return ticketCollectionData
    

In [8]:
#Variables pour faciliter l'écriture dans le cas où il faudrait changer le nom des fichiers
csvFilePath = 'organizers-data.csv'
scjsonFilePath = 'smart-contracts-data.json'
resultjsonFilePath = 'result.json'

allEvents = [] #Création de la list qui contiendra tous les events
datacsv = getdatacsv()

#Pour chaque event dans datacsv, on créer le dictionnaire avec toutes les informations qu'on insère dans la list allEvents
for i in range(len(datacsv)):
    datajson = getdatajson(datacsv[i+1]["id"])
    allEvents.append(remplirDic(createDictionnaire(),datacsv[i+1],createCollection(),datajson))

#Ecriture du fichier d'output en json
with open(resultjsonFilePath, 'w') as jsonFile: #Ecriture du fichier JSON avec les informations du fichier CSV
    jsonFile.write(json.dumps(allEvents,ensure_ascii= False, indent = 4))  
